<a href="https://colab.research.google.com/github/rahulvaidya286/MAD-1-L/blob/main/DL_Workshop_OPPE_24T3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Q1 Create a 3x5 tensor with random floating-point values between 0 and 1 after setting the seed to 42 using the torch library. Compute the sum of all elements in the tensor. (write the answer correct upto 2 decimals)

Ans: 9.96 (Range: 9.0 , 10)

In [ ]:
torch.manual_seed(42)
x = torch.rand(3, 5)
print(x)
torch.sum(x)

tensor([[0.8823, 0.9150, 0.3829, 0.9593, 0.3904],
        [0.6009, 0.2566, 0.7936, 0.9408, 0.1332],
        [0.9346, 0.5936, 0.8694, 0.5677, 0.7411]])


tensor(9.9613)

# Q2 what is the value of q3(z) in given below equation? where z = 1.57 (write the answer correct upto 2 decimals)

$$
q_1(z) = z^2\\
q_2(z) = z^3\\
q_3(z) = e^z* sin(z)\\
p(z) = \dfrac{q_1}{q_2} + q_3
$$

Ans: 4.80 (Range: 4.5, 5.1)

# Q3 what is the value of  $\dfrac{\partial p}{\partial z}$ ? where z = 1.57 (write the answer correct upto 2 decimals)

Ans: 4.40 (Range: 4.0, 4.8)

#Q4 what is the value of  $\dfrac{\partial p}{\partial q3}$ ? where z = 1.57 (write the answer correct upto 2 decimals)

Ans: 1 (Range: 0.7, 1.3)

In [ ]:
# required for the gradient tracking
Z = torch.tensor(1.57, requires_grad=True)
Z

tensor(1.5700, requires_grad=True)

In [ ]:
Z.is_leaf

True

In [ ]:
q1 = Z**2
q1.retain_grad()
q2 = Z**3
q3 = torch.exp(Z) * torch.sin(Z)
q3.retain_grad()
print(q1)
print(q2)
print(q3)

tensor(2.4649, grad_fn=<PowBackward0>)
tensor(3.8699, grad_fn=<PowBackward0>)
tensor(4.8066, grad_fn=<MulBackward0>)


In [ ]:
torch.exp(Z)

tensor(4.8066, grad_fn=<ExpBackward0>)

In [ ]:
torch.sin(Z)

tensor(1.0000, grad_fn=<SinBackward0>)

In [ ]:
p = (q1/q2) + q3
p

tensor(5.4436, grad_fn=<AddBackward0>)

In [ ]:
p.backward()

In [ ]:
Z.grad

tensor(4.4048)

In [ ]:
q1.grad

tensor(0.2584)

In [ ]:
1/q2

tensor(0.2584, grad_fn=<MulBackward0>)

In [ ]:
q3.grad

tensor(1.)

# Q5
Read the `mnist_train_small.csv` dataset provided by colab in sample_data folder. First column is the target column so accordingly divide the dataset into features and labels.

Before training scale the features using min-max-scaler
$$
\textit{X_scaled} = \dfrac{X - min}{max- min}
$$

- Use dtype = torch.float32 for feature tensor
- Use dtype = torch.long for label tensor

- Build the following network
  - Assume the input and output layer according to the data
  - first hidden layer with 128 neurons and Relu activation function
  - second hidden layer with 64 neurons and Relu activation function

- Initialize the model with torch.manual_seed(42)
- Use nn.CrossEntropyLoss() for loss calculation (this pytorch function internally applies softmax activation function before calculating the loss)

- Optimize the model using ADAM optimizer with learning_rate = 0.01 and train for 10 epochs

**Hint-** train the model multiple times if it's showing different initial loss values that means manual_seed(42) not set correctly

**Enter the number of parameters (weights & biases both) in the network.**

Ans: 109386

In [ ]:
df = pd.read_csv("/content/sample_data/mnist_train_small.csv",header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
mnist_X = df.drop(0,axis=1)
mnist_y= df[0]

In [ ]:
mnist_X.head()

,1,2,3,4,5,6,7,8,9,10,...,775,776,777,778,779,780,781,782,783,784
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Convert to tensors
X = torch.tensor(mnist_X.values, dtype=torch.float32)
y = torch.tensor(mnist_y.values, dtype=torch.long) # Assuming labels are integers

# Normalize the data
X = X / 255.0

In [ ]:
# Define the network architecture for MNIST
class MNISTClassifier(nn.Module):
  def __init__(self):
    super(MNISTClassifier, self).__init__()
    self.network = nn.Sequential(
        nn.Linear(784, 128),
        nn.ReLU(),
        nn.Linear(128, 64),
        nn.ReLU(),
        nn.Linear(64, 10)
    )

  def forward(self, x):
    return self.network(x)

In [ ]:
#@title train with sequential

# Initialize the model, loss function, and optimizer
torch.manual_seed(42)
model = MNISTClassifier()

criterion = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for classification
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
epochs = 10

for epoch in range(epochs):
  optimizer.zero_grad()
  outputs = model(X)
  loss = criterion(outputs, y)
  loss.backward()
  optimizer.step()
  print(f'Epoch {epoch +1}/{epochs}, Loss: {loss.item():.4f}')

print("Training Complete", loss.item())

Epoch 1/10, Loss: 2.3140
Epoch 2/10, Loss: 2.1494
Epoch 3/10, Loss: 1.8080
Epoch 4/10, Loss: 1.4193
Epoch 5/10, Loss: 1.0860
Epoch 6/10, Loss: 0.8703
Epoch 7/10, Loss: 0.7189
Epoch 8/10, Loss: 0.6853
Epoch 9/10, Loss: 0.5522
Epoch 10/10, Loss: 0.5768
Training Complete 0.5767546892166138


In [ ]:
num_params = 0
for param in model.parameters():
  num_params += param.numel()
print(f"Total number of parameters: {num_params}")

Total number of parameters: 109386


#Q6 What is the difference between the losses computed at last epoch and first epoch ? (correct answer upto two decimals)
 Ans: 1.7372 (1.66, 1.80)

In [ ]:
2.3140 - 0.5768

1.7372

In [ ]:
#@title without sequential
import pandas as pd
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn

torch.manual_seed(42)

# Convert to tensors
X = torch.tensor(mnist_X.values, dtype=torch.float32)
y = torch.tensor(mnist_y.values, dtype=torch.long) # Assuming labels are integers

# Normalize the data
X = X / 255.0


# Define the network architecture for MNIST
class MNISTClassifier(nn.Module):
    def __init__(self):
        super(MNISTClassifier, self).__init__()
        self.fc1 = nn.Linear(784, 128)  # Input layer: 784 features (28x28 pixels)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(64, 10)  # Output layer: 10 classes (digits 0-9)

    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.fc3(x)
        return x


# Initialize the model, loss function, and optimizer
model = MNISTClassifier()
criterion = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for classification
optimizer = optim.Adam(model.parameters(), lr=0.01)


# Training loop
epochs = 10

for epoch in range(epochs):
  optimizer.zero_grad()
  outputs = model(X)
  loss = criterion(outputs, y)
  loss.backward()
  optimizer.step()
  print(f'Epoch {epoch +1}/{epochs}, Loss: {loss.item():.4f}')

print("Training Complete")

Epoch 1/10, Loss: 2.3140
Epoch 2/10, Loss: 2.1494
Epoch 3/10, Loss: 1.8080
Epoch 4/10, Loss: 1.4193
Epoch 5/10, Loss: 1.0860
Epoch 6/10, Loss: 0.8703
Epoch 7/10, Loss: 0.7189
Epoch 8/10, Loss: 0.6853
Epoch 9/10, Loss: 0.5522
Epoch 10/10, Loss: 0.5768
Training Complete


# Q7 Design a CNN with one convolutional layer (16 filters, 5x5 kernel, stride = 2, padding = 1) followed by an average-pooling layer (2x2).

Use the below code to generate the random images
```python
img = torch.randint(0, 256, size=(128, 3, 32, 32), dtype=torch.float32)
```
what will be the output size after the average-pooling layer?

**Options**

(A) `torch.Size([128, 3, 64, 64])`

(B) `torch.Size([128, 16, 64, 15])`

(C) `torch.Size([128, 3, 32, 32])`

(D) `torch.Size([128, 16, 7, 7])`

**Answer**

D


In [ ]:
img = torch.randint(0, 256, size=(128, 3, 32, 32), dtype=torch.float32)
conv = nn.Conv2d(3, 16, kernel_size=5, stride=2, padding=1)
pool = nn.AvgPool2d(kernel_size=2)
out = conv(img)
out = pool(out)
result = out.size()
result

torch.Size([128, 16, 7, 7])

#Q8
Using the `torchvision.datasets` module, load the CIFAR-10 training dataset and use `torchvision.transforms` to convert them into tensors.

Use DataLoader to load the dataset with the following settings:
* Batch size = 128
* Shuffle = True

How many training data points are there in the last batch?

Ans: 80 No range is required

In [ ]:
import torch
import torchvision
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

transform = transforms.Compose([transforms.ToTensor()])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

100%|██████████| 170M/170M [00:14<00:00, 11.6MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data


In [ ]:
batch_size=128
train_dataset.data.shape[0] % batch_size

80

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

for batch in train_loader:
    last_batch = batch

last_batch_size = last_batch[0].shape
last_batch_size

torch.Size([80, 3, 32, 32])

#Q9 What is the size, in pixels, of a single image tensor in the CIFAR-10 dataset?
<!--3×32×32=3072 parameters (pixels).-->
ANS: 3072

In [ ]:
image, label = train_dataset[0]
print("Tensor shape:", image.shape)

Tensor shape: torch.Size([3, 32, 32])


In [ ]:
image.reshape(-1).shape

torch.Size([3072])

In [ ]:
torch.manual_seed(42)
torch.randint(0, 10, size=(4, 4), dtype=torch.float32)

tensor([[2., 7., 6., 4.],
        [6., 5., 0., 4.],
        [0., 3., 8., 4.],
        [0., 4., 1., 2.]])

In [ ]:
torch.manual_seed(42)
torch.randint(0, 3, size=(4,), dtype=torch.float32)

tensor([0., 2., 1., 1.])

#Q10 Use the following preprocessed dataset for training a simple RNN model:
```python
torch.manual_seed(42)
X_train = torch.randint(0, 10, size=(4, 4), dtype=torch.float32)

y_train = torch.randint(0, 8, size=(4,), dtype=torch.float32)
```

Use the manual seed to be 42.

Define and Train the RNN Model:

Model Structure:
RNN Layer: This layer will process the input sequences. The number of hidden units in the RNN layer is 5.

Fully Connected Layer: This layer will map the RNN output to the final output size.

Output Size: The output size is 1

Hidden Size: The hidden size (5)
Loss Function: Mean Squared Error (MSE) is used for regression tasks.

Optimizer: Stochastic Gradient Descent (SGD) is used with learning rate = 0.1.

Training: Train the model for 2 epochs.

Enter the loss after 2 epochs.


Ans: (Range: 18, 21)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim


torch.manual_seed(42)
X_train = torch.randint(0, 10, size=(4, 4), dtype=torch.float32)

y_train = torch.randint(0, 8, size=(4,), dtype=torch.float32)


class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x.unsqueeze(1))
        out = self.fc(out[:, -1, :])
        return out


input_size = X_train.size(1)
hidden_size = 5
output_size = 1

model = RNNModel(input_size, hidden_size, output_size)


criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)


epochs = 2
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = criterion(outputs.squeeze(), y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 2 == 0:
        print(f'Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}')


Epoch [2/2], Loss: 19.1193
